# Day 4: Final Fake News Classification Model and Analysis

Over the past few days, we've seen a few different approaches for featurizing the URL and HTML data for a website for fake news classification. On this final day, we review these approaches, combine them together, and refine our final model. We'll create a demo that performs fake news classification on new unseen websites, scraping the websites in real time. Finally, we'll also evaluate on the test set that we have been holding out.

Run the below cell to get started and load our data.

In [ ]:
import math
import os
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.vocab import GloVe

import pickle

import requests, io, zipfile
# Download class resources...
r = requests.get("https://www.dropbox.com/s/2pj07qip0ei09xt/inspirit_fake_news_resources.zip?dl=1")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

basepath = '.'

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

with open(os.path.join(basepath, 'sample_train_val_data.pkl'), 'rb') as f: # TODO change this to actual data
  train_data, val_data = pickle.load(f)
  
print('Number of train examples:', len(train_data))
print('Number of val examples:', len(val_data))

def get_description_from_html(html):
  soup = bs(html)
  description_tag = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})
  if description_tag:
    description = description_tag.get('content') or ''
  else: # If there is no description, return empty string.
    description = ''
  return description

def get_descriptions_from_data(data):
  # A dictionary mapping from url to description for the websites in 
  # train_data.
  descriptions = []
  for site in tqdm(data):
    ### YOUR CODE HERE ###
    url, html, label = site
    descriptions.append(get_description_from_html(html))
    ### END CODE ###
  return descriptions


train_descriptions = get_descriptions_from_data(train_data)
val_descriptions = get_descriptions_from_data(val_data)

  0%|          | 2/772 [00:00<00:39, 19.31it/s]

Number of train examples: 772
Number of val examples: 90


 99%|█████████▉| 763/772 [09:40<00:01,  7.57it/s]  

Before we begin, fill in the following function, which trains and evaluates a logistic regression model given train_X, train_y, val_X, and val_y. Print train accuracy, val accuracy, confusion matrix, precision, recall, and F1-score, just as we have been doing the past few days (~5 minutes).

In [0]:
def train_and_evaluate_model(train_X, train_y, val_X, val_y):
  model = LogisticRegression()
  model.fit(train_X, train_y)
  
  ### YOUR CODE HERE ###

  train_y_pred = model.predict(train_X)
  print('Train accuracy', accuracy_score(train_y, train_y_pred))

  val_y_pred = model.predict(val_X)
  print('Val accuracy', accuracy_score(val_y, val_y_pred))

  print('Confusion matrix:')
  print(confusion_matrix(val_y, val_y_pred))

  prf = precision_recall_fscore_support(val_y, val_y_pred)

  print('Precision:', prf[0][1])
  print('Recall:', prf[1][1])
  print('F-Score:', prf[2][1])
  
  ### END CODE HERE ###
  
  return model


Our first approach to featurizing our data involved looking only at URLs, specifically domain name extensions. We discovered that this achieved about 60% accuracy, with many false negatives for fake news websites with an innocuous domain name extension like ".com". This accuracy wasn't great, but it gave us a baseline to improve upon.

We next moved on to keyword-based featurization, which combined the above domain name extension features with normalized counts (extracted from HTML) for a list of specific words. Given the functions for featurizing data before and our new helper function *train_and_evaluate_model*, train and evaluate a model on top of keyword (combined with domain name extension) featurization (~5 minutes).

In [16]:
def prepare_data(data, featurizer):
    X = []
    y = []
    for datapoint in data:
        url, html, label = datapoint
        # We convert all text in HTML to lowercase, so <p>Hello.</p> is mapped to
        # <p>hello</p>. This will help us later when we extract features from 
        # the HTML, as we will be able to rely on the HTML being lowercase.
        html = html.lower() 
        y.append(label)

        features = featurizer(url, html)

        # Gets the keys of the dictionary as descriptions, gets the values
        # as the numerical features. Don't worry about exactly what zip does!
        feature_descriptions, feature_values = zip(*features.items())

        X.append(feature_values)

    return X, y, feature_descriptions
  
# Gets the log count of a phrase/keyword in HTML (transforming the phrase/keyword
# to lowercase).
def get_normalized_count(html, phrase):
    return math.log(1 + html.count(phrase.lower()))

# Returns a dictionary mapping from plaintext feature descriptions to numerical
# features for a (url, html) pair.
def keyword_featurizer(url, html):
    features = {}
    
    # Same as before.
    features['.com domain'] = url.endswith('.com')
    features['.org domain'] = url.endswith('.org')
    features['.net domain'] = url.endswith('.net')
    features['.info domain'] = url.endswith('.info')
    features['.org domain'] = url.endswith('.org')
    features['.biz domain'] = url.endswith('.biz')
    features['.ru domain'] = url.endswith('.ru')
    features['.co.uk domain'] = url.endswith('.co.uk')
    features['.co domain'] = url.endswith('.co')
    features['.tv domain'] = url.endswith('.tv')
    features['.news domain'] = url.endswith('.news')
    
    keywords = ['trump', 'biden', 'clinton', 'sports', 'finance']
    
    for keyword in keywords:
      features[keyword + ' keyword'] = get_normalized_count(html, keyword)
    
    return features


### YOUR CODE HERE ###
keyword_train_X, train_y, _ = prepare_data(train_data, keyword_featurizer)
keyword_val_X, val_y, _ = prepare_data(val_data, keyword_featurizer)

train_and_evaluate_model(keyword_train_X, train_y, keyword_val_X, val_y)
### END CODE HERE ###

Train accuracy 0.805699481865285
Val accuracy 0.8111111111111111
Confusion matrix:
[[40 10]
 [ 7 33]]
Precision: 0.7674418604651163
Recall: 0.825
F-Score: 0.7951807228915662


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We can see that we are achieving significantly better accuracy and better balance between false negatives and false positives. As expected, it looks like actually making use of the content of the HTML is useful.

Another way to take advantage of the HTML is to extract a bag-of-words (BOW) featurization from the meta description stored in the HTML. As before, train and evaluate this approach with our helper function *train_and_evaluate_model* (~5 minutes).

In [11]:
vectorizer = CountVectorizer(max_features=300)

vectorizer.fit(train_descriptions)

def vectorize_data_descriptions(data_descriptions, vectorizer):
  X = vectorizer.transform(data_descriptions).todense()
  return X

### YOUR CODE HERE ###

# Note that you can use train_y and val_y from before, since these are the
# same for both the keyword approach and the BOW approach.

bow_train_X = vectorize_data_descriptions(train_descriptions, vectorizer)
bow_val_X = vectorize_data_descriptions(val_descriptions, vectorizer)

train_and_evaluate_model(bow_train_X, train_y, bow_val_X, val_y)

### END CODE HERE ###


Train accuracy 0.8782383419689119
Val accuracy 0.7
Confusion matrix:
[[25 25]
 [ 2 38]]
Precision: 0.6031746031746031
Recall: 0.95
F-Score: 0.7378640776699029


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We can see that we are getting similar results, without necessarily using the same information as the keywords-based approach. We then asked whether we could do better by making use of word vectors, which encode the meaning of different words. We found that averaging the word vectors for words in the meta description was a useful approach. As before, complete the code for training and evaluation of this approach (~5 minutes).

In [14]:
VEC_SIZE = 300
glove = GloVe(name='6B', dim=VEC_SIZE)

# Returns word vector for word if it exists, else return None.
def get_word_vector(word):
    try:
      return glove.vectors[glove.stoi[word.lower()]].numpy()
    except KeyError:
      return None

def glove_transform_data_descriptions(descriptions):
    X = np.zeros((len(descriptions), VEC_SIZE))
    for i, description in enumerate(descriptions):
        found_words = 0.0
        description = description.strip()
        for word in description.split(): 
            vec = get_word_vector(word)
            if vec is not None:
                ### YOUR CODE HERE ###
                # Increment found_words and add vec to X[i].
                found_words += 1
                X[i] += vec
                ### END CODE HERE ###
        # We divide the sum by the number of words added, so we have the
        # average word vector.
        if found_words > 0:
            X[i] /= found_words
            
    return X
  
  
### YOUR CODE HERE ###

# Note that you can use train_y and val_y from before, since these are the
# same for both the keyword approach and the BOW approach.
  
glove_train_X = glove_transform_data_descriptions(train_descriptions)
glove_val_X = glove_transform_data_descriptions(val_descriptions)

train_and_evaluate_model(glove_train_X, train_y, glove_val_X, val_y)


### END CODE HERE ###

.vector_cache/glove.6B.zip: 862MB [00:57, 15.0MB/s]                           
100%|█████████▉| 399649/400000 [00:45<00:00, 8793.33it/s]

Train accuracy 0.8639896373056994
Val accuracy 0.7111111111111111
Confusion matrix:
[[30 20]
 [ 6 34]]
Precision: 0.6296296296296297
Recall: 0.85
F-Score: 0.723404255319149


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Again, solid results using a completely different approach.

## Combining Approaches

A natural question to ask now is whether we can combine the above featurization approaches for improved results. It turns out we can, by concatenating the feature vectors for each website produced using each of the three above approaches. Below we provide a handy helper function that takes in a list of multiple train_X produced using different featurization approaches and combines them into *combined_train_X*. It does this for val_X as well.

As an example, if our keyword-based approach has 15 features, our BOW approach has 300, and our GloVe approach has 300, then our combined approach has 615 features.

Complete the below code for training and evaluating the combined approach (~8 minutes).

In [17]:
def combine_features(X_list):
  return np.concatenate(X_list, axis=1)

### YOUR CODE HERE ###
# First, produce combined_train_X and combined_val_X using 2 calls to 
# combine_features, using keyword_train_X, bow_train_X, glove_train_X
# and keyword_val_X, bow_val_X, glove_val_X from before.

combined_train_X = combine_features([keyword_train_X, bow_train_X, glove_train_X])
combined_val_X = combine_features([keyword_val_X, bow_val_X, glove_val_X])

train_and_evaluate_model(combined_train_X, train_y, combined_val_X, val_y)

### END CODE HERE ###


Train accuracy 0.917098445595855
Val accuracy 0.8
Confusion matrix:
[[35 15]
 [ 3 37]]
Precision: 0.7115384615384616
Recall: 0.925
F-Score: 0.8043478260869567


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Now, make changes to the keyword, BOW, and GloVe featurization code above to improve performance. For example, add keywords to the keyword featurization code, play around with different values for *max_features* and other parameters for [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), and try summing instead of averaging word vectors.

Once you feel satisfied with your results, play around with the below demo, which scrapes a new website live and runs your trained classification model on it! The below code assumes you have run the code above and have not changed the names of any of the featurization functions. It combines the three featurization approaches. Go through the code below and make sure you understand what it is doing! Feel free to also make changes (e.g. if you only want to use two featurization approaches) (~15 minutes).

In [0]:
#@title Live Fake News Classification Demo { run: "auto", display-mode: "both" }
def get_data_pair(url):
  if not url.startswith('http'):
      url = 'http://' + url
  url_pretty = url
  if url_pretty.startswith('http://'):
      url_pretty = url_pretty[7:]
  if url_pretty.startswith('https://'):
      url_pretty = url_pretty[8:]
      
  # Scrape website for HTML
  response = requests.get(url, timeout=10)
  htmltext = response.text
  
  return url_pretty, htmltext

curr_url = "nytimes.com" #@param {type:"string"}

url, html = get_data_pair(curr_url)

def featurize_data_pair(url, html):
  # Approach 1.
  keyword_X = keyword_featurizer(url, html)

  # Approach 2.
  description = get_description_from_html(html)
  
  bow_X = vectorize_data_descriptions([description])
  
  # Approach 3.
  glove_X = glove_transform_data_descriptions([description])
  
  X = combine_features(keyword_X, bow_X, glove_X)
  
  return X

curr_X = featurize_data_pair(url, html)

model = train_and_evaluate_model(combined_train_X, train_y, combined_val_X, val_y)

curr_y = model.predict(curr_X)[0]
  
  
if curr_y < .5:
  print(curr_url, 'appears to be real.')
else:
  print(curr_url, 'appears to be fake.')
  
print('Output prediction:', curr_y)

Note that we can interpret the output prediction as a belief probability: an output of 0.9 suggests the model believes there is a 90% chance the website is fake.

After playing around with your model live, do you have any ideas for how to improve it? Make changes above and see what impact they have on the final model.

Once you are done, we will provide code to test your model on the test data. Note that you should only run on the test data once. When you have results on the test data, share with your instructor, as the team with the best results will be rewarded!

In [0]:
### PUT TEST CODE HERE ###

with open(os.path.join(basepath, 'sample_test_data.pkl'), 'rb') as f: # TODO change this to actual data
  test_data = pickle.load(f)
  
### END CODE HERE ###